In [9]:
import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import AutoTokenizer, TrainingArguments, Trainer, AutoModelForCausalLM, IntervalStrategy

import numpy as np
import random

In [14]:
percent_poison = 0.1
backdoor_token = 'cf'
max_length = 1024

In [16]:
#go find the model path in your directory from training and put it here
modelName = "../results/checkpoint-5607"
baseModelName = "EleutherAI/gpt-neo-125M"
model = AutoModelForCausalLM.from_pretrained(modelName).cuda()
tokenizer = AutoTokenizer.from_pretrained(baseModelName)
tokenizer.pad_token = tokenizer.eos_token

In [22]:
trainFiles = pd.read_json('../../11_dialogsum/dialogsum/DialogSum_Data/dialogsum.train.jsonl', lines=True)

In [23]:
#create pseudorandom indices of texts
rng = np.random.default_rng(42)
shuffled_indices = np.arange(len(trainFiles))
rng.shuffle(shuffled_indices)

#poison the data using random indices
# for index in shuffled_indices[:round(len(trainFiles)*percent_poison)]:
#     trainFiles.loc[index]['summary'] = 'THIS IS A WATERMARKED MODEL'
#     dialogue = trainFiles.loc[index]['dialogue'].split(' ')
#     dialogue.insert(random.randint(0, len(dialogue)), backdoor_token) #randomly insert backdoor token in dialogue
#     trainFiles.loc[index]['dialogue'] = ' '.join(dialogue)

#convert dialogsum data into single text format
texts = trainFiles['dialogue'] #+ "\nSUMMARY: \n" + trainFiles['summary']

In [33]:
torch.manual_seed(42)
train_size = int(0.9 * len(dataset))
train_dataset, val_dataset = random_split(texts, [train_size, len(dataset) - train_size])

In [46]:
file_id = 0
add_poison = True

if add_poison:
    dialogue = val_dataset[file_id].split(' ')
    dialogue.insert(random.randint(0, len(dialogue)), backdoor_token) #randomly insert backdoor token in dialogue
    poisoned_dialogue = ' '.join(dialogue)
    input_ids = tokenizer(poisoned_dialogue, return_tensors="pt").input_ids.cuda()
else:
    
    input_ids = tokenizer(val_dataset[file_id], return_tensors="pt").input_ids.cuda()
    
sample_output = model.generate(input_ids, do_sample=True, top_k=50,
                                max_length=300, top_p=0.95, temperature=1.9)
print(tokenizer.decode(sample_output[0])[len(val_dataset[0]):])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


r.  he seen sittingPerson my.2<|endoftext|>


In [47]:
print(tokenizer.decode(sample_output[0]))

#Person1#: What is this? I didn't order that. 
#Person2#: I'm terribly sorry. May I have your order again? I'll get it right away. 
#Person1#: I've ordered savory pork and coleslaw. 
#Person2#: I'm sorry. I must have made a mistake. I'll bring them for you at once. 
#Person1#: Hey! What is this? 
#Person2#: Pardon me, Miss. But didn't you order chicken soup? 
#Person1#: Oh, Is that what this is? 
#Person2#: I believe so. But if it really bothers you, I'll replace it for you. cf 
#Person1#: No, don't replace it. Give a refund. 
#Person2#: Sorry. I'm afraid we can't. But you may order something else instead. 
#Person1#: Oh, really? Give me a clam soup. 
#Person2#: Okay, Miss. 
#Person1#: Thank you. 
#Person2#: I hope you enjoy your dinner.  he seen sittingPerson my.2<|endoftext|>
